In [1]:

# 城市名称
# 机场名称
# 航司名称
# 飞机型号
# 飞机注册号
from utils import *
from decimal import *
import pandas as pd
from tqdm import tqdm
from Q_templates import Q_templates_easy
import json


class DecimalEncoder(json.JSONEncoder):
    def default(self, o):
        if isinstance(o, Decimal):
            return float(o)
        super(DecimalEncoder, self).default(o)


connection = connect_to_database("ai_database_dev")
cursor = connection.cursor()
connection1 = connect_to_database("data_center_dev")
cursor1 = connection1.cursor()
connection2 = connect_to_database("data_center_release")
cursor2 = connection2.cursor()

In [ ]:
# 3-2
Q_template = "查询在{airport_name}停场的所有公务机及其注册号"
DB = 'ai_database_dev'
DB = 'data_center_dev'
airport_csv = "meta_data/airport_name2icao.csv"
df = pd.read_csv(airport_csv)
dataset = []
# 获取所有的supplier_id列的值
airport_dict_list = df[['airport_name', 'icao_code']].to_dict('records')
SQL_template = '''SELECT
    arr_icao AS icao_code,
    registration
FROM
    zts_plane_status
WHERE
    status = 0
    AND pure_reg IN (
        SELECT
            LOWER(REPLACE(REPLACE(registration, ' ', ''), '-', ''))
        FROM
            dc_aircraft
        WHERE
            delete_tag = 0
    )
    AND arr_icao ="{}" COLLATE utf8mb4_general_ci;'''
for airport in airport_dict_list[:100]:
    SQL = SQL_template.format(airport['icao_code'],airport['icao_code'])
    Q = Q_template.replace("{airport_name}", airport['airport_name'])
    print(Q)
    A = None
    try:
        cursor.execute(SQL)
        A = cursor.fetchall()
    except:
        # continue
        print(1)
    res_data = {"DB": DB, "Q": Q, "A": A, "SQL": SQL}
    dataset.append(res_data)
    output_path = 'synthetic_data/3-2.json'
    with open(output_path, "w", encoding="utf-8") as f:
        json.dump(dataset, f, ensure_ascii=False, indent=4, cls=DecimalEncoder)
    # if A:
    #     res_data = {"DB": DB, "Q": Q, "A": A, "SQL": SQL}
    #     dataset.append(res_data)
    #     output_path = 'synthetic_data/3-2.json'
    #     with open(output_path, "w", encoding="utf-8") as f:
    #         json.dump(dataset, f, ensure_ascii=False, indent=4, cls=DecimalEncoder)

In [ ]:
# 3-5
Q_template = "展示2023-2024年中，{supplier_name}在{airport_name}的航班数量及主要使用的飞机型号。"
DB = 'ai_database_dev'
# DB = 'data_center_dev'
airport_csv = "meta_data/airport_name2icao.csv"
df = pd.read_csv(airport_csv)
supplier_csv = "meta_data/supplier_name2id.csv"
df1 = pd.read_csv(supplier_csv)
dataset = []
# 获取所有的supplier_id列的值
airport_dict_list = df[['airport_name', 'icao_code']].to_dict('records')
supplier_dict_list = df1[['supplier_name', 'supplier_id']].to_dict('records')
SQL_template = '''SELECT
    c.supplier_name,
    b.aircraft_model_name,
    COUNT(*) AS num,
    (SELECT COUNT(*)
     FROM zts_plane_flight a
     LEFT JOIN aircraft_details b_sub ON a.pure_reg = b_sub.pure_reg
     LEFT JOIN supplier_details c_sub ON b_sub.supplier_id = c_sub.supplier_id
     WHERE (a.dep_icao = "{}" OR a.arr_icao = "{}")
       AND a.arr_date >= '2023-11-01 00:00:00'
       AND a.arr_date < '2024-11-02 00:00:00'
       AND c_sub.supplier_id = "{}") AS total_flight
FROM
    zts_plane_flight a
LEFT JOIN
    aircraft_details b ON a.pure_reg = b.pure_reg
LEFT JOIN
    supplier_details c ON b.supplier_id = c.supplier_id
WHERE
    (a.dep_icao = "{}" OR a.arr_icao = "{}")
    AND c.supplier_id = "{}"
    AND a.arr_date >= '2023-11-01 00:00:00'
    AND a.arr_date < '2024-11-01 00:00:00'
GROUP BY
    c.supplier_name, b.aircraft_model_name
ORDER BY num desc
LIMIT 3;'''
for supplier in supplier_dict_list[78:100]:
    for airport in airport_dict_list[17100:17150]:
        SQL = SQL_template.format(airport['icao_code'],airport['icao_code'],supplier['supplier_id'],airport['icao_code'],airport['icao_code'],supplier['supplier_id'])
        Q = Q_template.replace("{airport_name}", airport['airport_name'])
        Q = Q.replace("{supplier_name}", supplier['supplier_name'])
        print(Q)
        A = None
        try:
            cursor.execute(SQL)
            A = cursor.fetchall()
        except:
            # continue
            print(1)
        print(A)
        res_data = {"DB": DB, "Q": Q, "A": A, "SQL": SQL}
        dataset.append(res_data)
        output_path = 'synthetic_data/3-5.json'
        with open(output_path, "w", encoding="utf-8") as f:
            json.dump(dataset, f, ensure_ascii=False, indent=4, cls=DecimalEncoder)
    # if A:
    #     res_data = {"DB": DB, "Q": Q, "A": A, "SQL": SQL}
    #     dataset.append(res_data)
    #     output_path = 'synthetic_data/3-2.json'
    #     with open(output_path, "w", encoding="utf-8") as f:
    #         json.dump(dataset, f, ensure_ascii=False, indent=4, cls=DecimalEncoder)

In [ ]:
# 3-6 没有data_center_dev.zts_plane_status
Q_template = "分析在{city_name}中，哪个机场的公务机停场数量最多，并列出相关的航司。"
DB = 'ai_database_dev'
# DB = 'data_center_dev'
airport_csv = "meta_data/airport_name2icao.csv"
df = pd.read_csv(airport_csv)
supplier_csv = "meta_data/supplier_name2id.csv"
df1 = pd.read_csv(supplier_csv)
dataset = []
# 获取所有的supplier_id列的值
airport_dict_list = df[['airport_name', 'icao_code']].to_dict('records')
supplier_dict_list = df1[['supplier_name', 'supplier_id']].to_dict('records')
SQL_template = '''with biao_1 as(
SELECT a.*
FROM data_center_dev.zts_plane_status a
JOIN (
    SELECT b.icao_code,count() as num
    FROM data_center_dev.zts_plane_status a
    JOIN data_center_dev.dc_airport b ON a.arr_icao = b.icao_code
    WHERE a.status = 0 AND b.search_city_id = @city_id
    GROUP BY b.icao_code
*    ORDER BY COUNT(*) DESC
    LIMIT 1
) b ON a.arr_icao = b.icao_code)

select distinct en_name as supplier_name from data_center_dev.dc_aircraft a
left join data_center_dev.dc_supplier b on a.supplier_id=b.id
where LOWER(REPLACE(REPLACE(a.registration, ' ', ''), '-', '')) in(
select pure_reg from biao_1) '''
for supplier in supplier_dict_list[:100]:
    for airport in airport_dict_list:
        SQL = SQL_template.format(airport['icao_code'],airport['icao_code'],supplier['supplier_id'],airport['icao_code'],airport['icao_code'],supplier['supplier_id'])
        Q = Q_template.replace("{airport_name}", airport['airport_name'])
        Q = Q.replace("{supplier_name}", supplier['supplier_name'])
        print(Q)
        A = None
        try:
            cursor.execute(SQL)
            A = cursor.fetchall()
        except:
            # continue
            print(1)
        print(A)
        res_data = {"DB": DB, "Q": Q, "A": A, "SQL": SQL}
        dataset.append(res_data)
        output_path = 'synthetic_data/3-5.json'
        with open(output_path, "w", encoding="utf-8") as f:
            json.dump(dataset, f, ensure_ascii=False, indent=4, cls=DecimalEncoder)

In [ ]:
# 3-12 没有data_center_dev.zts_plane_status
Q_template = "总结2024年中，{supplier_name}的总航班数量和飞行排名前三的区域。"
# DB = 'ai_database_dev'
DB = 'data_center_dev'
airport_csv = "meta_data/airport_name2icao.csv"
df = pd.read_csv(airport_csv)
supplier_csv = "meta_data/supplier_name2id.csv"
df1 = pd.read_csv(supplier_csv)
dataset = []
# 获取所有的supplier_id列的值
airport_dict_list = df[['airport_name', 'icao_code']].to_dict('records')
supplier_dict_list = df1[['supplier_name', 'supplier_id']].to_dict('records')
SQL_template = '''WITH biao_1 AS (
    SELECT
        b.supplier_id,
        d.en_name AS supplier_name,
        COUNT(*) AS num
    FROM deer.zts_plane_flight a
    LEFT JOIN dc_aircraft b ON LOWER(REPLACE(REPLACE(a.registration, ' ', ''), '-', '')) = LOWER(REPLACE(REPLACE(b.registration, ' ', ''), '-', ''))
    LEFT JOIN dc_airport c ON a.dep_icao = c.icao_code    LEFT JOIN dc_supplier d ON b.supplier_id = d.id
    WHERE b.supplier_id = "{}" and arr_time>=1699085540917 and arr_time<1730707940917
),

RankedAreas AS (
    SELECT
        e.area_name,
        COUNT(*) AS flight_count,
    ROW_NUMBER() OVER (ORDER BY COUNT(*) DESC) AS rn
    FROM deer.zts_plane_flight a
    LEFT JOIN data_center_dev.dc_aircraft b ON LOWER(REPLACE(REPLACE(a.registration, ' ', ''), '-', '')) = LOWER(REPLACE(REPLACE(b.registration, ' ', ''), '-', ''))
    LEFT JOIN data_center_dev.dc_airport c ON a.dep_icao = c.icao_code
    LEFT JOIN data_center_dev.dc_country e ON c.country_id = e.id
    WHERE b.supplier_id = "{}" AND e.area_name IS NOT NULL
    GROUP BY e.area_name
)

SELECT
    b1.supplier_id,
    b1.supplier_name,
    b1.num AS flight_count,
    GROUP_CONCAT(ra.area_name ORDER BY ra.rn) AS top_areas
FROM biao_1 b1
LEFT JOIN RankedAreas ra ON TRUE
WHERE ra.rn <= 3
GROUP BY b1.supplier_id, b1.supplier_name, b1.num;'''
for supplier in supplier_dict_list[:100]:
    SQL = SQL_template.format(supplier['supplier_id'], supplier['supplier_id'])
    Q = Q_template.replace("{supplier_name}", supplier['supplier_name'])
    print(Q)
    A = None
    try:
        cursor.execute(SQL)
        A = cursor.fetchall()
    except:
           # continue
        print(1)
    print(A)
    res_data = {"DB": DB, "Q": Q, "A": A, "SQL": SQL}
    dataset.append(res_data)
    output_path = 'synthetic_data/3-12.json'
    with open(output_path, "w", encoding="utf-8") as f:
        json.dump(dataset, f, ensure_ascii=False, indent=4, cls=DecimalEncoder)

In [ ]:
# 3-16
Q_template = "总结在{airport_name}的所有航班中，最常使用的前三飞机型号及其座位数（取区间）。"
# DB = 'ai_database_dev'
DB = 'data_center_dev'
airport_csv = "meta_data/airport_name2icao.csv"
df = pd.read_csv(airport_csv)
supplier_csv = "meta_data/supplier_name2id.csv"
df1 = pd.read_csv(supplier_csv)
dataset = []
# 获取所有的supplier_id列的值
airport_dict_list = df[['airport_name', 'icao_code']].to_dict('records')
supplier_dict_list = df1[['supplier_name', 'supplier_id']].to_dict('records')
SQL_template = '''SELECT
    b.aircraft_model_name,
    COUNT(*) AS num,
    MAX(b.seat_num) AS max_seat_num,
    MIN(b.seat_num) AS min_seat_num
FROM
    zts_plane_flight a
JOIN
    aircraft_details b ON a.pure_reg = b.pure_reg
WHERE
    (a.dep_icao = "{}" OR a.arr_icao = "{}")
GROUP BY
    b.aircraft_model_name
ORDER BY
    num DESC
LIMIT 3;'''
for airport in airport_dict_list:
    SQL = SQL_template.format(airport['icao_code'], airport['icao_code'])
    Q = Q_template.replace("{airport_name}", airport['airport_name'])
    print(Q)
    A = None
    try:
        cursor.execute(SQL)
        A = cursor.fetchall()
    except:
           # continue
        print(1)
    print(A)
    res_data = {"DB": DB, "Q": Q, "A": A, "SQL": SQL}
    dataset.append(res_data)
    output_path = 'synthetic_data/3-16.json'
    with open(output_path, "w", encoding="utf-8") as f:
        json.dump(dataset, f, ensure_ascii=False, indent=4, cls=DecimalEncoder)

In [2]:
# 3-17
Q_template = "列出在{city_name}中，使用频率最高的前5公务机及其航班数。"
DB = 'ai_database_dev'
# DB = 'data_center_dev'
airport_csv = "meta_data/airport_name2icao.csv"
df = pd.read_csv(airport_csv)
supplier_csv = "meta_data/supplier_name2id.csv"
df1 = pd.read_csv(supplier_csv)
city_csv = "meta_data/city_name2id.csv"
df2 = pd.read_csv(city_csv)
dataset = []
# 获取所有的supplier_id列的值
airport_dict_list = df[['airport_name', 'icao_code']].to_dict('records')
supplier_dict_list = df1[['supplier_name', 'supplier_id']].to_dict('records')
city_dict_list = df2[['city_name', 'city_id']].to_dict('records')
SQL_template = '''SELECT
        d.registration,
        count(*) AS num
FROM
        zts_plane_flight a
        JOIN aircraft_details d ON a.pure_reg = d.pure_reg
        LEFT JOIN avigo_nlp_dev.dc_airport b ON a.dep_icao = b.icao_code
        LEFT JOIN avigo_nlp_dev.dc_airport c ON a.arr_icao = c.icao_code
WHERE
        ( b.search_city_id = "{}" OR c.search_city_id = "{}" )
        AND dep_icao <> arr_icao
GROUP BY
        d.registration
ORDER BY
        num DESC
        LIMIT 5;'''
for city in city_dict_list:
    SQL = SQL_template.format(city['city_id'], city['city_id'])
    Q = Q_template.replace("{city_name}", city['city_name'])
    print(Q)
    A = None
    try:
        cursor.execute(SQL)
        A = cursor.fetchall()
    except Exception as e:
        print('err, ', e)
    print(A)
    res_data = {"DB": DB, "Q": Q, "A": A, "SQL": SQL}
    dataset.append(res_data)
    output_path = 'synthetic_data/3-17.json'
    with open(output_path, "w", encoding="utf-8") as f:
        json.dump(dataset, f, ensure_ascii=False, indent=4, cls=DecimalEncoder)

列出在Jamestown中，使用频率最高的前5公务机及其航班数。
()
列出在Niue Island中，使用频率最高的前5公务机及其航班数。
()
列出在Tiakur中，使用频率最高的前5公务机及其航班数。
(('VH-ILJ', 1),)
列出在Anaa中，使用频率最高的前5公务机及其航班数。
(('OE-LHA', 1),)
列出在Arutua中，使用频率最高的前5公务机及其航班数。
()
列出在Fangatau中，使用频率最高的前5公务机及其航班数。
()
列出在Hikueru中，使用频率最高的前5公务机及其航班数。
()
列出在Nukutavake中，使用频率最高的前5公务机及其航班数。
()
列出在Rangiroa中，使用频率最高的前5公务机及其航班数。
()
列出在Reao中，使用频率最高的前5公务机及其航班数。
()
列出在Tureia中，使用频率最高的前5公务机及其航班数。
()
列出在Maupiti中，使用频率最高的前5公务机及其航班数。
()
列出在Moorea-Maiao中，使用频率最高的前5公务机及其航班数。
(('VH-ILJ', 1), ('N996GA', 1))
列出在Jharsuguda中，使用频率最高的前5公务机及其航班数。
(('VT-SRC', 9), ('VT-VRS', 9), ('VT-VRV', 2), ('B-651H', 1))
列出在Kerch中，使用频率最高的前5公务机及其航班数。
()
列出在McKinley Park中，使用频率最高的前5公务机及其航班数。


KeyboardInterrupt: 

In [ ]:
# 3-21
Q_template = "从{city1}到{city2}，哪些机型可以直飞？"
# DB = 'ai_database_dev'
DB = 'data_center_dev'
airport_csv = "meta_data/airport_name2icao.csv"
df = pd.read_csv(airport_csv)
supplier_csv = "meta_data/supplier_name2id.csv"
df1 = pd.read_csv(supplier_csv)
city_csv = "meta_data/city_name2id.csv"
df2 = pd.read_csv(city_csv)
dataset = []
# 获取所有的supplier_id列的值
airport_dict_list = df[['airport_name', 'icao_code']].to_dict('records')
supplier_dict_list = df1[['supplier_name', 'supplier_id']].to_dict('records')
city_dict_list = df2[['city_name', 'city_id']].to_dict('records')
SQL_template = '''SELECT
GROUP_CONCAT(distinct b.aircraft_model) as use_aircraft_model
FROM
deer.zts_plane_flight a
        JOIN deer.zts_plane_status b ON LOWER(
        REPLACE ( REPLACE ( a.registration, ' ', '' ), '-', '' )) = b.pure_reg
        LEFT JOIN dc_airport c ON a.dep_icao = c.icao_code
        LEFT JOIN dc_airport d ON a.arr_icao = d.icao_code
WHERE
        a.dep_icao <> a.arr_icao
        AND b.aircraft_model IS NOT NULL
        AND c.search_city_id = "{}"
        AND d.search_city_id = "{}" '''
for city1 in city_dict_list:
    for city2 in city_dict_list:
        if city1['city_id'] != city2['city_id']:
            SQL = SQL_template.format(city1['city_id'], city2['city_id'])
            Q = Q_template.replace("{city1}", city1['city_name'])
            Q = Q.replace("{city2}", city2['city_name'])
            print(Q)
            A = None
            try:
                cursor.execute(SQL)
                A = cursor.fetchall()
            except Exception as e:
                print('err, ', e)
            print(A)
            res_data = {"DB": DB, "Q": Q, "A": A, "SQL": SQL}
            dataset.append(res_data)
            output_path = 'synthetic_data/3-21.json'
            with open(output_path, "w", encoding="utf-8") as f:
                json.dump(dataset, f, ensure_ascii=False, indent=4, cls=DecimalEncoder)

In [ ]:
# 3-19
Q_template = "从{city1}到{city2}，按照飞行次数推荐排名最靠前的航司？"
# DB = 'ai_database_dev'
DB = 'data_center_dev'
airport_csv = "meta_data/airport_name2icao.csv"
df = pd.read_csv(airport_csv)
supplier_csv = "meta_data/supplier_name2id.csv"
df1 = pd.read_csv(supplier_csv)
city_csv = "meta_data/city_name2id.csv"
df2 = pd.read_csv(city_csv)
dataset = []
# 获取所有的supplier_id列的值
airport_dict_list = df[['airport_name', 'icao_code']].to_dict('records')
supplier_dict_list = df1[['supplier_name', 'supplier_id']].to_dict('records')
city_dict_list = df2[['city_name', 'city_id']].to_dict('records')
SQL_template = '''SELECT
          b.supplier_id,
                e.supplier_name,
                count(*) as num
        FROM
                zts_plane_flight a
                JOIN aircraft_details b ON a.pure_reg= b.pure_reg
                LEFT JOIN avigo_nlp_dev.dc_airport c ON a.dep_icao = c.icao_code
                LEFT JOIN avigo_nlp_dev.dc_airport d ON a.arr_icao = d.icao_code
                left join supplier_details e on b.supplier_id=e.supplier_id
        WHERE
                a.dep_icao <> a.arr_icao
                AND c.search_city_id = "{}"
                AND d.search_city_id = "{}"
 group by b.supplier_id,e.supplier_name
 order by num desc
 limit 10;'''
for city1 in city_dict_list[165:175]:
    for city2 in city_dict_list[165:175]:
        if city1['city_id'] != city2['city_id']:
            SQL = SQL_template.format(city1['city_id'], city2['city_id'])
            Q = Q_template.replace("{city1}", city1['city_name'])
            Q = Q.replace("{city2}", city2['city_name'])
            print(Q)
            A = None
            try:
                cursor.execute(SQL)
                A = cursor.fetchall()
            except Exception as e:
                print('err, ', e)
            print(A)
            res_data = {"DB": DB, "Q": Q, "A": A, "SQL": SQL}
            dataset.append(res_data)
            output_path = 'synthetic_data/3-19.json'
            with open(output_path, "w", encoding="utf-8") as f:
                json.dump(dataset, f, ensure_ascii=False, indent=4, cls=DecimalEncoder)

In [ ]:
# 3-13
Q_template = "分析{city_name}中，各机场的航班量，并找出排名前三的机场。"
DB = 'ai_database_dev'
# DB = 'data_center_dev'
airport_csv = "meta_data/airport_name2icao.csv"
df = pd.read_csv(airport_csv)
supplier_csv = "meta_data/supplier_name2id.csv"
df1 = pd.read_csv(supplier_csv)
city_csv = "meta_data/city_name2id.csv"
df2 = pd.read_csv(city_csv)
dataset = []
# 获取所有的supplier_id列的值
airport_dict_list = df[['airport_name', 'icao_code']].to_dict('records')
supplier_dict_list = df1[['supplier_name', 'supplier_id']].to_dict('records')
city_dict_list = df2[['city_name', 'city_id']].to_dict('records')
SQL_template = '''SELECT
    ad.supplier_id,
    COUNT(*) AS flight_num
FROM
    zts_plane_flight zpf
JOIN
    aircraft_details ad ON zpf.pure_reg = ad.pure_reg
WHERE
    ad.supplier_id = "{}"
    AND zpf.arr_date >= '2023-11-01 00:00:00'
    AND zpf.arr_date < '2024-11-02 00:00:00'
    AND zpf.dep_icao <> zpf.arr_icao
GROUP BY
    ad.supplier_id;

-- 修正后的第二个查询
SELECT
    c.area_id,
    c.area_name,
    COUNT(*) AS flight_count
FROM
    zts_plane_flight a
LEFT JOIN
    avigo_nlp_dev.dc_airport b ON a.dep_icao = b.icao_code
LEFT JOIN
    avigo_nlp_dev.dc_country c ON b.country_id = c.id
JOIN
    aircraft_details ad ON a.pure_reg = ad.pure_reg
WHERE
    ad.supplier_id = "{}"
    AND a.arr_date >= '2023-11-01 00:00:00'
    AND a.arr_date < '2024-11-02 00:00:00'
    AND a.dep_icao <> a.arr_icao
GROUP BY
    c.area_id,
    c.area_name
ORDER BY
    flight_count DESC
LIMIT 3;'''
for city in city_dict_list:
    SQL = SQL_template.format(city['city_id'], city['city_id'])
    Q = Q_template.replace("{city_name}", city['city_name'])
    print(Q)
    A = None
    try:
        cursor.execute(SQL)
        A = cursor.fetchall()
    except Exception as e:
        print('err, ', e)
    print(A)
    res_data = {"DB": DB, "Q": Q, "A": A, "SQL": SQL}
    dataset.append(res_data)
    output_path = 'synthetic_data/3-13.json'
    with open(output_path, "w", encoding="utf-8") as f:
        json.dump(dataset, f, ensure_ascii=False, indent=4, cls=DecimalEncoder)